In [52]:
import xlsxwriter
from bs4 import BeautifulSoup
from pprint import pprint
import urllib2
import json
import sys
import collections
import datetime
import warnings
warnings.filterwarnings('ignore')

In [53]:
# Create an new Excel file and add a worksheet.
workbook = xlsxwriter.Workbook('Haemophilus.xlsx')
worksheet = workbook.add_worksheet()

In [54]:
def validate(date_text):
    try:
        datetime.datetime.strptime(date_text, '%Y-%m-%d')
        return True
    except ValueError:
        return False
        raise ValueError("Incorrect data format, should be YYYY-MM-DD")

def parser(baslangic_yil, bitis_yil):
    #db = MySQLdb.connect("localhost","root","5462593180","wiki_twitter_google")
    lines = open('wiki_article_name.txt').read().splitlines()
    row = 0
    col = 0
    temp = 0
    for line in lines:
        worksheet.write(row,col, str(line))
        col = col + 1
        temp = col
        row = 0
        for yil in range(int(baslangic_yil),int(bitis_yil)+1):
            for ay in range(1,13):
                #print str(ay) + " " + str(yil)
                url_line = line.replace(" ","%20")
                if ay < 10:
                    ay = "0" + str(ay)
                url = "http://stats.grok.se/json/en/"+str(yil)+str(ay)+"/"+url_line
                page = urllib2.urlopen(url)
                soup = BeautifulSoup(page.read())
                context = json.loads(soup.get_text() )
                day_list = context['daily_views']
                if len(day_list) > 0:
                    day_list = collections.OrderedDict(sorted(day_list.items()))
                    for day in day_list:
                        if(validate(day)):
                            row += 1
                            col = temp
                            worksheet.write(row,col,day)
                            col += 1
                            worksheet.write(row,col,day_list[day])
                                #sql = "INSERT INTO wiki (article_name, date_, view) VALUES ('%s', '%s', '%d')" % (line,day,day_list[day])
                                #cursor.execute(sql)
        
    workbook.close()

    #f = open(sys.argv[1],'r')
    #out = f.read().splitlines() # will append in the list out

In [55]:
parser(2010,2015)